In [1]:
# whether to log each feature and sequence status
verbose = 1

In [2]:
import os
import pandas as pd
import numpy as np
import json
from PIL import Image
import cv2
from sklearn.utils import shuffle
import sys
sys.path.append('..')

import h5py
import random

In [3]:
import keras

Using TensorFlow backend.


In [115]:
# import pretrained model functions
from deepvideoclassification.models import precompute_CNN_features
from deepvideoclassification.models import load_pretrained_model_preprocessor
from deepvideoclassification.models import load_pretrained_model

# import pretrained model properties
from deepvideoclassification.models import pretrained_model_len_features
from deepvideoclassification.models import pretrained_model_sizes
from deepvideoclassification.models import pretrained_model_names, poolings

In [5]:
# setup paths
pwd = os.getcwd().replace("deepvideoclassification","")
path_cache = pwd + 'cache/'
path_data = pwd + 'data/'

In [6]:
# setup logging
# any explicit log messages or uncaught errors to stdout and file /logs.log
import logging
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s [%(threadName)-12.12s] [%(levelname)-5.5s]  %(message)s",
    handlers=[
        logging.FileHandler("{0}/{1}.log".format(pwd, "logs")),
        logging.StreamHandler()
    ])
# init logger
logger = logging.getLogger()
# make logger aware of any uncaught exceptions
def handle_exception(exc_type, exc_value, exc_traceback):
    if issubclass(exc_type, KeyboardInterrupt):
        sys.__excepthook__(exc_type, exc_value, exc_traceback)
        return

    logger.error("Uncaught exception", exc_info=(exc_type, exc_value, exc_traceback))
sys.excepthook = handle_exception

In [7]:
# read vid folders
def get_video_paths():
    """
    Return list of video paths 

    Videos should be in /data/video_1/, /data/video_2/ style folders 
    with sequentially numbered frame images e.g. /data/video_1/frame00001.jpg

    There should be at least 3 videos, 1 for each of train/test/valid splits
    Split assignment is given in /data/labels.csv (also to be provided by user)

    Functionality to use different parts of a video as train/valid/test 
    not currently implemented.
    """
    path_videos = []
    for filename in os.listdir(path_data):
        if os.path.isdir(os.path.join(path_data, filename)):
            path_videos.append(filename)

    path_videos = [path_data + v + '/' for v in path_videos]

    # make sure that there is video data in /data/ and give instructions if not done correctly
    assert len(path_videos)>0, "There need to be at least 3 video folders (at least 1 for each of train, valid, \
    and test splits) in /data/ - each video should be its own folder of frame images with ascending time-ordered \
    filenames e.g. /data/vid1/frame00001.jpg ... videos assignment to train/valid/test split should be given in \
    /data/labels.csv ... cross-validation or train/valid/test splits within a single long video not currently implemented"

    return path_videos

In [8]:
def resize_frames(target_size):
    """
    Resize the frames of all videos and save them to /cache/ 
    to make model fitting faster .

    We resize once upfront rather than each we use a pretrained model or architecture.

    Our models require inputs resized to:
    * 224 x 224 VGG16, ResNet50, DenseNet, MobileNet
    * 299 x 299 XCeption, InceptionV3, InceptionResNetV2
    * 112 x 112 3D CNN 
    """

    if not os.path.exists(path_cache + 'frames/' + str(target_size[0]) + "_" + str(target_size[1]) + '/'):
        
        os.makedirs(path_cache + 'frames/' + str(target_size[0]) + "_" + str(target_size[1]) + '/')

        # read vid paths
        path_videos = get_video_paths()

        # loop over all vids and resize frames, saving to new folder in /cache/frames/
        for c,path_video in enumerate(path_videos):

            logger.info("resizing vid {}/{} to {}x{}".format(c+1,len(path_videos),target_size[0], target_size[1]))

            # get vid name from path
            video_name = path_video.split("/")[-2]

            # create directory for resized frames - just storing arrays now so commented out
            # e.g. path_vid_resized = /cache/frames/224_224/s23-4847/
            # path_vid_resized = path_cache + 'frames/'
            # path_vid_resized += str(target_size[0]) + "_" + str(target_size[1]) + '/' 
            # path_vid_resized += video_name + '/'

            # load frame paths for vid
            path_frames = os.listdir(path_video)
            path_frames = [path_video + f for f in path_frames if f != '.DS_Store']
            path_frames.sort()

            # load frames
            frames = []
            for path_frame in path_frames:

                # open image and resize
                filename = path_frame.split("/").pop()
                img_frame = Image.open(path_frame)
                img_frame = img_frame.resize(target_size)
                # img_frame.save(path_vid_resized + filename, "JPEG", quality = 100)

                # convert to array and append to list
                img_frame = np.array(img_frame)
                frames.append(img_frame)

            # save array of resized frames
            np.save(path_cache + "frames/" + str(target_size[0]) + "_" + str(target_size[1]) + "/" + video_name, np.array(frames))

In [9]:
def get_labels():
    # read labels - should be CSV with columns "video","frame","label","split"
    # e.g. "s1-218", "s1-218-00001.jpeg", "noseal", "train"
    labels = None
    try:
        labels = pd.read_csv(path_data + 'labels.csv', usecols=['video','frame','label','split'])
    except ValueError as e:
        raise Exception("Labels file must contain columns ['video','frame','label','split'] - if you only have ['video','frame','label'], use Jupyter notebook in notebooks/add_splits_to_labels_file.ipynb to add train/valid/test splits to your labels file")
    except FileNotFoundError as e:
        raise Exception("No labels found - please save labels file to /data/labels.csv") from e

    return labels.sort_values(["video","frame"])

In [10]:
def create_video_label_arrays():
    """
    Create numpy array with labels for each vid and a label_map.json file
    in /cache/labels/
    """

    # create folder for labels
    if not os.path.exists(path_cache + 'labels/'):
        os.makedirs(path_cache + 'labels/')

    # load labels
    labels = get_labels()

    # build label_map
    label_dummies = pd.get_dummies(labels, columns = ['label'])

    # get label columns list and build label map dict
    label_columns = []
    label_map = {}
    label_map_idx = 0
    for i, col in enumerate(label_dummies.columns):
        if col[:6] == 'label_':
            label_columns.append(col)
            label_map[label_map_idx] = col
            label_map_idx+=1

    # save label map to json
    with open(path_cache + 'labels/label_map.json', 'w') as fp:
        json.dump(label_map, fp)

    # get video paths
    path_videos = get_video_paths()

    # save numpy array of labels for each vid
    for path_video in path_videos:

        # get vid name from path
        video_name = path_video.split("/")[-2]

        vid_labels = np.array(label_dummies[label_dummies['video'] == video_name][label_columns])

        # save labels array for this vid
        np.save(path_cache + "/labels/" + video_name, np.array(vid_labels))

In [11]:
def load_label_map():
    """
    Returns label map - read from disk
    """

    # load label map from disk
    label_map = None
    try:
        if os.path.exists(path_cache + 'labels/label_map.json'):
            with open(path_cache + 'labels/label_map.json', 'r') as fp:
                label_map = json.load(fp)
        else:
            # build labels and label map
            create_video_label_arrays()
            if os.path.exists(path_cache + 'labels/label_map.json'):
                with open(path_cache + 'labels/label_map.json', 'r') as fp:
                    label_map = json.load(fp)
    except Exception as e:
        logger.error ('label map not found - make sure /data/labels.csv exists and data cache has been built')

    return label_map

In [12]:
class DataGenerator(keras.utils.Sequence):
    """
    Generator (used in Data) that generates data for Keras fit_generator method because full dataset too big to load into memory
    
    > inherits from keras.utils.Sequence
    
    """
    def __init__(self, batch_size, h5_path, h5_row_count):
        """
        Initialization DataGenerator class
        
        :batch_size: number of samples to return in batch 
        :h5_path: path to h5 dataset (where we stored the generated sequence data via save_frame_sequences_to_h5())
        :h5_row_count: number of rows in h5 dataset
        
        """
        self.batch_size = batch_size
        self.h5_path = h5_path
        self.h5_row_count = h5_row_count
        
        # init (shuffle dataset)
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(self.h5_row_count / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        
        # Generate indexes of the batch
        batch_indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Generate data
        x, y = self.__data_generation(batch_indexes)

        return x, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(self.h5_row_count)
        
        # shuffle indexes -> shuffle samples returned in each batch
        np.random.shuffle(self.indexes)

    def __data_generation(self, batch_indexes):
        """
        Generates data containing batch_size samples
        
        :batch_indexes: list (of size batch_size) with indexes into h5 file
        """ 
        x, y = None, None

        # slices into h5 file need to be sorted
        batch_indexes.sort()

        # read sample from h5 file
        with h5py.File(self.h5_path, 'r') as h5:
            ### read sample indexes from h5 file
            # sample sequences
            x = h5['sequences'][batch_indexes,:]
            # sample labels
            y = h5['labels'][batch_indexes,:]

        return x, y 

In [107]:
class Data(object):
    
    def __init__(self, sequence_length, 
                    return_CNN_features = False, pretrained_model_name = None, pooling = None, 
                    frame_size = None, augmentation = False, oversampling = False,
                    model_weights_path = None, custom_model_name = None,
                    return_generator = False, batch_size = None,
                    verbose = True):
        """
        Data object constructor
        
        
        :sequence_length: number of frames in sequence to be returned by Data object
        :return_CNN_features: whether to return precomputed features or return frames (or sequences of features/frames if sequence_length>1)

        :return_features: if True then return features (or sequences of feature) from pretrained model, if False then return frames (or sequences of frames)        
        :pretrained_model_name: name of pretrained model (or None if not using pretrained model e.g. for 3D-CNN)
        :pooling: name of pooling variant (or None if not using pretrained model e.g. for 3D-CNN)
        :frame_size: size that frames are resized to (this is looked up for pretrained models)
        :augmentation: whether to apply data augmentation (horizontal flips)
        :oversampling: whether to apply oversampling to create class balance
        
        :model_weights_path: path to custom model weights if we want to load CNN model we've fine-tuned to produce features (e.g. for LRCNN)
        :custom_model_name: custom output name to append to pretrained model name
        
        :return_generator: if True and sequence_length > 1 and return_CNN_features == False, then do not return dataset, instead construct h5 file with sequences for each split and return generator that samples from that (dataset of sequecne frames too big to load into memory)
        :batch_size: size of batches that generator must return
        
        :verbose: whether to log details
        
        Notes: 
        * if pretrained_model_name != None and return_CNN_features=False then will first apply preprocessor to frames (or frame sequences)
        * if return_generator = True and sequence_length > 1 and return_CNN_features == False, large h5 files will be created in cache before returning generator
        """
    
        # required params
        self.sequence_length = sequence_length
        self.frame_size = frame_size
        
        # optional params
        self.pretrained_model_name = pretrained_model_name
        self.pooling = pooling
        self.return_CNN_features = return_CNN_features
        self.augmentation = augmentation
        self.oversampling = oversampling
        
        # init model data
        self.x_train = []
        self.y_train = []
        #
        self.x_valid = []
        self.y_valid = []
        # 
        self.x_test = []
        self.y_test = []
        
        # fix case sensitivity
        if type(self.pretrained_model_name) == str:
            self.pretrained_model_name = self.pretrained_model_name.lower()
        if type(self.pooling) == str:
            self.pooling = self.pooling.lower()
        
        ################
        ### Prepare data
        ################
        
        # get video paths
        self.path_videos = get_video_paths()
        
        # create label array for each video and load label map
        create_video_label_arrays()
        self.label_map = load_label_map()
        
        # get labels
        self.labels = get_labels()
        
        # check that there is 1 frame file for each label file and raise error if they don't match
        paths_frames = []
        for folder, subs, files in os.walk(path_data):        
            for filename in files:
                if filename[-4:].lower() == '.jpg' or filename[-4:].lower() == 'jpeg' or filename[-4:].lower() == 'png':
                    paths_frames.append(os.path.abspath(os.path.join(folder, filename)))
        if len(paths_frames) != len(self.labels):
            error_msg = 'IMPORTANT ERROR: Number of frames ({}) in /data/ video folders needs to match number of labels ({}) in labels.csv - use notebooks/helper_check_frames_against_labels.ipynb to investigate... Note, only labels.csv and the frames you want to use (in video subfolders) should be in /data/'.format(len(paths_frames), len(self.labels))
            logger.info(error_msg)
            raise ValueError(error_msg)
        
        # pull number of classes from labels shape
        self.num_classes = self.labels['label'].nunique()
        
        # create dict mapping video to train/valid/test split assignment
        video_splits = self.labels[['video','split']].drop_duplicates()
        video_splits.set_index("video", inplace=True)
        video_splits = video_splits.to_dict()['split']
        self.video_splits = video_splits
        
        # look up target size for pretrained model
        if pretrained_model_name is not None:
            self.frame_size = pretrained_model_sizes[pretrained_model_name]
        
        # precompute resized frames (won't recompute if already resized)
        resize_frames(self.frame_size)

        # pre compute CNN features (won't recompute if already computed)
        if return_CNN_features and pretrained_model_name is not None:
            # check if pass custom weights to precompute from
            if model_weights_path is not None and custom_model_name is not None:
                # precompute with custom weights input and name
                precompute_CNN_features(self.pretrained_model_name, self.pooling, self.model_weights_path, self.custom_model_name)
            else:
                precompute_CNN_features(self.pretrained_model_name, self.pooling)
            
            
            
        # get preprocessor given pretrained if we will need to apply preprocessor 
        # (i.e. if return_CNN_features == False and pretrained_model_name != None)
        if not return_CNN_features and pretrained_model_name is not None:
            self.preprocess_input = load_pretrained_model_preprocessor(self.pretrained_model_name)
        
        
        self.verbose = verbose
        
        self.return_generator = return_generator
        self.batch_size = batch_size
        
        # do some checks
        if self.return_generator:
            assert self.batch_size != None, "batch size required to construct generator"
        if self.return_generator:
            assert self.return_CNN_features == False, "generator only implemented for frame sequences - features usually large enough to load into memory [may take a few minutes]"
        
        ###################################
        ### load features / build sequences
        ###################################
        
        
        # load features/frames from all videos and concat into big array for each of train, valid and test
        if self.sequence_length > 1:
            
            ### sequences
            
            if return_CNN_features:
                
                if verbose:
                    logging.info("Loading features sequence data into memory [may take a few minutes]")
                
                #####################
                ### feature sequences
                #####################
                
                path_features = path_cache + 'features/' + pretrained_model_name + "/" + pooling + '/'
                if not return_CNN_features and pretrained_model_name is not None:
                    path_features = path_cache + 'features/' + pretrained_model_name + "__" + custom_model_name + "/" + pooling + '/'
                path_labels = path_cache + 'labels/'
                
                # read vid paths
                path_videos = get_video_paths()

                # loop over all vids and load precomputed features into memory as sequences
                for c, path_video in enumerate(path_videos):

                    if verbose:
                        logging.info("Loading features sequence data into memory {}/{}".format(c+1,len(path_videos)))
                    
                    # get vid name from path
                    video_name = path_video.split("/")[-2]
                    
                    ### create sequence: features
                    # load precomputed features
                    features = np.load(path_features + video_name + '.npy')
                    # build sequences
                    x = []
                    for i in range(sequence_length, len(features) + 1):
                        x.append(features[i-sequence_length:i])
                    x = np.array(x)
                    
                    
                    ### create sequence: labels
                    # load precomputed labels
                    labels = np.load(path_labels + video_name + '.npy')     

                    # temp lists to store sequences
                    y = []
                    for i in range(sequence_length, len(labels) + 1):
                        y.append(labels[i-1])
                    y = (np.array(y))
                    
                    ### build output
                    if self.video_splits[video_name] == "train":
                        self.x_train.append(x)
                        self.y_train.append(y)
                    if self.video_splits[video_name] == "valid":
                        self.x_valid.append(x)
                        self.y_valid.append(y)
                    if self.video_splits[video_name] == "test":
                        self.x_test.append(x)
                        self.y_test.append(y)
                        
            else:

                ###################
                ### frame sequences
                ###################
                
                # load full frame sequecne dataset into memory and return
                if not return_generator:
                    
                    ##############################################################################
                    ### load full sequence dataset into memory (will likely run into memory error)
                    ##############################################################################
                    
                    if verbose:
                        logging.info("Loading frame sequence data into memory [may take a few minutes]")

                    # load resized numpy array
                    path_vid_resized = path_cache + 'frames/'
                    path_vid_resized += str(self.frame_size[0]) + "_" + str(self.frame_size[1]) + '/' 

                    path_labels = path_cache + 'labels/'

                    # read vid paths
                    path_videos = get_video_paths()

                    # loop over all vids and load full frame sequences into memory 
                    # (recommend using generator if lots of data to avoid out of memory issues)
                    for c, path_video in enumerate(path_videos):
                        
                        if verbose:
                            logging.info("Loading frame sequence data into memory {}/{}".format(c+1,len(path_videos)))

                        # get vid name from path
                        video_name = path_video.split("/")[-2]

                        ### create sequence: features
                        # load precomputed frames
                        frames = np.load(path_vid_resized  + video_name + '.npy')

                        # first apply preprocessing if pretrained model given
                        if pretrained_model_name != None:
                            frames = self.preprocess_input(frames.astype(np.float32))

                        # build sequences
                        x = []
                        for i in range(sequence_length, len(frames) + 1):
                            x.append(frames[i-sequence_length:i])
                        x = np.array(x)


                        ### create sequence: labels
                        # load precomputed labels
                        labels = np.load(path_labels + video_name + '.npy')     

                        # temp lists to store sequences
                        y = []
                        for i in range(sequence_length, len(labels) + 1):
                            y.append(labels[i-1])
                        y = (np.array(y))

                        ### build output
                        if self.video_splits[video_name] == "train":
                            self.x_train.append(x)
                            self.y_train.append(y)
                        if self.video_splits[video_name] == "valid":
                            self.x_valid.append(x)
                            self.y_valid.append(y)
                        if self.video_splits[video_name] == "test":
                            self.x_test.append(x)
                            self.y_test.append(y)
                else:
                    #############################
                    ### Build sequences generator
                    #############################
            
                    # compute and save h5 sequence files (save_frame_sequences_to_h5 returns 
                    # the sequence sizes which we need for our generator)
                    self.total_rows_train, self.total_rows_valid, self.total_rows_test = self.save_frame_sequences_to_h5()
                    
                    # init generators
                    self.generator_train = DataGenerator(self.batch_size, self.path_h5_train, self.total_rows_train)
                    self.generator_valid = DataGenerator(self.batch_size, self.path_h5_valid, self.total_rows_valid)
                    self.generator_test = DataGenerator(self.batch_size, self.path_h5_test, self.total_rows_test)
                
        else:

            ### not sequence
            
            if return_CNN_features:
                
                if verbose:
                    logging.info("Loading features data into memory [may take a few minutes]")
                
                ###################
                ### feature vectors
                ###################
                
                path_features = path_cache + 'features/' + pretrained_model_name + "/" + pooling + '/'
                if not return_CNN_features and pretrained_model_name is not None:
                    path_features = path_cache + 'features/' + pretrained_model_name + "__" + custom_model_name + "/" + pooling + '/'
                
                path_labels = path_cache + 'labels/'
                
                # read vid paths
                path_videos = get_video_paths()

                # loop over all vids and load precomputed features
                for c, path_video in enumerate(path_videos):

                    if verbose:
                        logging.info("Loading features data into memory: {}/{}".format(c+1,len(path_videos)))
                    
                    # get vid name from path
                    video_name = path_video.split("/")[-2]
                    
                    ### load precomputed features
                    x = np.load(path_features + video_name + '.npy')
                    y = np.load(path_labels + video_name + '.npy')
                    
                    if x.shape[0] != y.shape[0]:
                        print("XXX", path_video, x.shape, y.shape)
                    
                    ### build output
                    if self.video_splits[video_name] == "train":
                        self.x_train.append(x)
                        self.y_train.append(y)
                    if self.video_splits[video_name] == "valid":
                        self.x_valid.append(x)
                        self.y_valid.append(y)
                    if self.video_splits[video_name] == "test":
                        self.x_test.append(x)
                        self.y_test.append(y)
            else:
                
                if verbose:
                    logging.info("Loading frames into memory [may take a few minutes]")
                
                #################
                ### single frames
                #################
                
                # load resized numpy array
                path_vid_resized = path_cache + 'frames/'
                path_vid_resized += str(self.frame_size[0]) + "_" + str(self.frame_size[1]) + '/' 
                
                path_labels = path_cache + 'labels/'
                
                # read vid paths
                path_videos = get_video_paths()

                # loop over all vids and load frames into memory
                for c, path_video in enumerate(path_videos):

                    if verbose:
                        logging.info("Loading frames into memory: {}/{}".format(c+1,len(path_videos)))
                    
                    # get vid name from path
                    video_name = path_video.split("/")[-2]
                    
                    # load precomputed numpy arrays for frames and labels
                    x = np.load(path_vid_resized  + video_name + '.npy')
                    y = np.load(path_labels + video_name + '.npy')
                    
                    # apply preprocessing if pretrained model given
                    if pretrained_model_name != None:
                        x = self.preprocess_input(x.astype(np.float32))
                
                    ### build output
                    if self.video_splits[video_name] == "train":
                        self.x_train.append(x)
                        self.y_train.append(y)
                    if self.video_splits[video_name] == "valid":
                        self.x_valid.append(x)
                        self.y_valid.append(y)
                    if self.video_splits[video_name] == "test":
                        self.x_test.append(x)
                        self.y_test.append(y)
            
            
        #################################
        ### get file paths for each split
        #################################
        #
        # Note: only makes sense for sequence_length = 1
        
        # get file paths: train
        dflab = self.labels[self.labels['split'] == 'train']
        self.paths_train = list(path_data + dflab['video'] + "/" + dflab['frame'])

        # get file paths: valid
        dflab = self.labels[self.labels['split'] == 'valid']
        self.paths_valid = list(path_data + dflab['video'] + "/" + dflab['frame'])

        # get file paths: test
        dflab = self.labels[self.labels['split'] == 'test']
        self.paths_test = list(path_data + dflab['video'] + "/" + dflab['frame'])
        
        
        
        #################################################
        ### reshape list outputs (if not using generator)
        #################################################
        
        if not return_generator:
            ## e.g. (9846, 224, 224, 3) for frames [return_CNN_features=True]
            ## or  (9846, 512) for features [return_CNN_features=False]
            self.x_train = np.concatenate(self.x_train, axis=0)
            self.y_train = np.concatenate(self.y_train, axis=0)
            self.x_valid = np.concatenate(self.x_valid, axis=0)
            self.y_valid = np.concatenate(self.y_valid, axis=0)
            self.x_test = np.concatenate(self.x_test, axis=0)
            self.y_test = np.concatenate(self.y_test, axis=0)
            
            # shuffle train and validation set
            self.shuffle()
            

    def __str__(self):
        return "x_train: {}, y_train: {} ... x_valid: {}, y_valid: {} ... x_test: {}, y_test: {}".format(self.x_train.shape,self.y_train.shape,self.x_valid.shape,self.y_valid.shape,self.x_test.shape,self.y_test.shape)
            
    def shuffle(self):
        """
        Randomize the order of samples in train and valid splits
        """
        ###########
        ### shuffle
        ###########
        print(self.x_train.shape, self.y_train.shape, len(self.paths_train))
        if self.sequence_length == 1:
            self.x_train, self.y_train, self.paths_train = shuffle(self.x_train, self.y_train, self.paths_train)
            self.x_valid, self.y_valid, self.paths_valid = shuffle(self.x_valid, self.y_valid, self.paths_valid)
        else:
            self.x_train, self.y_train = shuffle(self.x_train, self.y_train)
            self.x_valid, self.y_valid = shuffle(self.x_valid, self.y_valid)

        

    # Even at small sequence lengths, loading the full dataset as 
    # a sequence into memory is not feasible so we need to use generators
    # that iterate over the dataset without loading it all into memory
    # 
    # For now, we will assume that we will load the features datasets into memory
    # because this is more feasible but for large datasets, we'd want to use generators for that too. 
    # An implementation for a feature generator  can be done by pattern matching the implementation for frames 
    # 
    # we first precompute a sequences h5 file (it's too big to fit in memory but we never have more than 1
    # video's sequences in memory) ...then we will initialize a generator that samples sequences from the 
    # h5 file and returns batches that will be passed to our model's fit_generator method

    def save_frame_sequences_to_h5(self):
        """
        Save sequence of frames to h5 files (1 for each split) in cache 
        because dataset too big to load into memory
        
        Will create generator that reads random rows from these h5 files
        
        Inspired by: https://stackoverflow.com/questions/41849649/write-to-hdf5-and-shuffle-big-arrays-of-data
        """
    
        #######################
        ### setup h5 file paths
        #######################
        
        if not os.path.exists(path_cache + 'sequences/'):
            os.makedirs(path_cache + 'sequences/')

        path_h5_base = path_cache + 'sequences/'

        # store h5 files in subfolder in cache/sequences/ either with pretrained model name or resize name
        # since we need to run preprocessing for pretrained models but not for vanilla resizing (3DCNN)
        if pretrained_model_name is not None:
            path_h5_base += pretrained_model_name + '/'
        else:
            path_h5_base += str(self.frame_size[0]) + "_" + str(self.frame_size[1]) + '/' 

        if not os.path.exists(path_h5_base):
            os.makedirs(path_h5_base)

        self.path_h5_train = path_h5_base + 'h5_' + str(self.sequence_length) + '_train.h5'
        self.path_h5_valid = path_h5_base + 'h5_' + str(self.sequence_length) + '_valid.h5'
        self.path_h5_test = path_h5_base + 'h5_' + str(self.sequence_length) + '_test.h5'
    
        # build h5 file if doesn't exists()
        if not os.path.exists(self.path_h5_train) or not os.path.exists(self.path_h5_valid) or not os.path.exists(self.path_h5_test) or not os.path.exists(path_h5_base + 'h5_meta.json'):
            
            # delete partially created cache
            paths_to_clear = [self.path_h5_train, self.path_h5_valid, self.path_h5_test, path_h5_base + 'h5_meta.json']
            for path_to_clear in paths_to_clear:                
                if os.path.exists(path_to_clear):
                    print("Removing partially created sequences cache file: {}".format(path_to_clear))
                    os.remove(path_to_clear)
             
            if verbose:
                logging.info("Computing frame sequence h5 files: {} [may take a few minutes]".format(path_h5_base))

            ##################################################
            ### get size of train/valid/test sequence datasets
            ##################################################

            # total number of rows of sequence data we have for each split
            # this is not the same as the number of frames since we exclude
            # the first (self.sequence_length-1) frames
            total_rows_train = 0
            total_rows_valid = 0
            total_rows_test = 0

            # load resized numpy array
            path_vid_resized = path_cache + 'frames/'
            path_vid_resized += str(self.frame_size[0]) + "_" + str(self.frame_size[1]) + '/' 

            path_labels = path_cache + 'labels/'

            # read vid paths
            path_videos = get_video_paths()

            # loop over all vids and resize frames, saving to new folder in /cache/frames/
            for c, path_video in enumerate(path_videos):
                                
                if verbose:
                    logging.info("Computing frame sequence h5 files: {}/{} [precompute]".format(c+1,len(path_videos)))

                # get vid name from path
                video_name = path_video.split("/")[-2]

                # load resized frames
                frames = np.load(path_vid_resized  + video_name + '.npy')

                # build sequences
                x = []
                for i in range(self.sequence_length, len(frames) + 1):
                    x.append(frames[i-self.sequence_length:i])
                x = np.array(x)

                if self.video_splits[video_name] == "train":
                    total_rows_train += len(x)
                if self.video_splits[video_name] == "valid":
                    total_rows_valid += len(x)
                if self.video_splits[video_name] == "test":
                    total_rows_test += len(x)

            # calc shapes required for full sequence dataset
            h5_shape_train_x = (total_rows_train, self.sequence_length, self.frame_size[0], self.frame_size[1], 3)
            h5_shape_train_y = (total_rows_train, self.num_classes)

            h5_shape_valid_x = (total_rows_valid, self.sequence_length, self.frame_size[0], self.frame_size[1], 3)
            h5_shape_valid_y = (total_rows_valid, self.num_classes)

            h5_shape_test_x = (total_rows_test, self.sequence_length, self.frame_size[0], self.frame_size[1], 3)
            h5_shape_test_y = (total_rows_test, self.num_classes)


            ################################
            ### Initialize and open h5 files
            ################################

            # open h5 file to store big sequence dataset feature and label arrays
            # path_h5file = MODEL -> SEQUENCE LENGTH
            f_train = h5py.File(self.path_h5_train, 'a')
            f_valid = h5py.File(self.path_h5_valid, 'a')
            f_test = h5py.File(self.path_h5_test, 'a')

            # initialize h5 datasets
            h5_train_x = f_train.create_dataset('sequences', shape= h5_shape_train_x, dtype='uint8')
            h5_train_y = f_train.create_dataset('labels', shape= h5_shape_train_y, dtype='uint8')

            h5_valid_x = f_valid.create_dataset('sequences', shape= h5_shape_valid_x, dtype='uint8')
            h5_valid_y = f_valid.create_dataset('labels', shape= h5_shape_valid_y, dtype='uint8')

            h5_test_x = f_test.create_dataset('sequences', shape= h5_shape_test_x, dtype='uint8')
            h5_test_y = f_test.create_dataset('labels', shape= h5_shape_test_y, dtype='uint8')

            ##################################################
            ### Build h5 files for this sequence / model combo
            ##################################################

            # load resized numpy array
            path_vid_resized = path_cache + 'frames/'
            path_vid_resized += str(self.frame_size[0]) + "_" + str(self.frame_size[1]) + '/' 

            path_labels = path_cache + 'labels/'

            # read vid paths
            path_videos = get_video_paths()

            # keep track of where we are in the h5 file
            h5_cursor_train = 0
            h5_cursor_valid = 0
            h5_cursor_test = 0

            # loop over all vids and build sequences file
            for c, path_video in enumerate(path_videos):
                
                if verbose:
                    logging.info("Computing frame sequence h5 files: {}/{} [build h5 file]".format(c+1,len(path_videos)))

                # get vid name from path
                video_name = path_video.split("/")[-2]

                ### create sequence: features
                # load precomputed frames
                frames = np.load(path_vid_resized  + video_name + '.npy')
                
                # first apply preprocessing if pretrained model given
                if pretrained_model_name != None:
                    frames = self.preprocess_input(frames.astype(np.float32))
                    
                # build sequences
                x = []
                for i in range(self.sequence_length, len(frames) + 1):
                    x.append(frames[i-self.sequence_length:i])
                x = np.array(x)

                ### create sequence: labels
                # load precomputed labels
                labels = np.load(path_labels + video_name + '.npy')     

                # temp lists to store sequences
                y = []
                for i in range(self.sequence_length, len(labels) + 1):
                    y.append(labels[i-1])
                y = (np.array(y))

                ### write this vid's data to relevant h5 dataset
                if self.video_splits[video_name] == "train":
                    h5_train_x[h5_cursor_train:h5_cursor_train + x.shape[0], :, :, :, :] = x
                    h5_train_y[h5_cursor_train:h5_cursor_train + y.shape[0], :] = y
                    h5_cursor_train += len(x)
                if self.video_splits[video_name] == "valid":
                    h5_valid_x[h5_cursor_valid:h5_cursor_valid + x.shape[0], :, :, :, :] = x
                    h5_valid_y[h5_cursor_valid:h5_cursor_valid + y.shape[0], :] = y
                    h5_cursor_valid += len(x)
                if self.video_splits[video_name] == "test":
                    h5_test_x[h5_cursor_test:h5_cursor_test + x.shape[0], :, :, :, :] = x
                    h5_test_y[h5_cursor_test:h5_cursor_test + y.shape[0], :] = y
                    h5_cursor_test += len(x)
            
            # save total row counts to file
            with open(path_h5_base + 'h5_meta.json', 'w') as fp:
                json.dump({'total_rows_train':total_rows_train,
                           'total_rows_valid': total_rows_valid,
                           'total_rows_test':total_rows_test}
                          , fp)
                    
            # close h5 files
            f_train.close()
            f_valid.close()
            f_test.close()
        
            # return total samples for each split so we can pass them to our DataGenerator
            return total_rows_train, total_rows_valid, total_rows_test
        
        else:
            # h5 sequence file already exists - just load the sequence meta and return sequence lengths 
            # so we can pass them to our DataGenerator
            total_rows_train, total_rows_valid, total_rows_test = None, None, None
            
            with open(path_h5_base + 'h5_meta.json', 'r') as fp:
                h5_meta = json.load(fp)
                total_rows_train = h5_meta['total_rows_train']
                total_rows_valid = h5_meta['total_rows_valid']
                total_rows_test = h5_meta['total_rows_test']
                
            # return total samples for each split so we can pass them to our DataGenerator
            return total_rows_train, total_rows_valid, total_rows_test

# TEST GENERATOR

## setup

In [32]:
from deepvideoclassification.models import Architecture
from deepvideoclassification.models import train

In [33]:
pretrained_model_name = "vgg16"
pooling="max"
sequence_length = 3

layer_1_size = 128
layer_2_size = 64
layer_3_size = 32
dropout=0.20

## train with no generator

In [ ]:
data = Data(sequence_length = sequence_length, 
            return_CNN_features = False, 
            pretrained_model_name=pretrained_model_name,
            pooling = pooling)

num_classes = data.num_classes 
frame_size = data.frame_size
num_features = pretrained_model_len_features[pretrained_model_name]

In [ ]:
architecture = Architecture(architecture="video_LRCNN_trainable", 
                            sequence_model = 'LSTM',
                            sequence_model_layers = 1,
                            sequence_length = sequence_length,
                            num_classes = data.num_classes, 
                            frame_size = data.frame_size, 
                            pretrained_model_name='vgg16', 
                            pooling='max',
                            layer_1_size=64, 
                            layer_2_size=32, 
                            layer_3_size=8, 
                            dropout=0.2,
                            convolution_kernel_size=3)

In [ ]:
# train model with no data generator
train(architecture.model, data, path_model = pwd+'models/', learning_rate = 0.001, epochs = 3)

## train with generator

In [ ]:
data = Data(sequence_length = sequence_length, 
            return_CNN_features = False, 
            pretrained_model_name=pretrained_model_name,
            pooling = pooling,
            batch_size=32,
            return_generator=True)

In [ ]:
from keras.optimizers import Adam

In [ ]:
architecture = Architecture(architecture="video_LRCNN_trainable", 
                            sequence_model = 'LSTM',
                            sequence_model_layers = 1,
                            sequence_length = sequence_length,
                            num_classes = data.num_classes, 
                            frame_size = data.frame_size, 
                            pretrained_model_name='vgg16', 
                            pooling='max',
                            layer_1_size=64, 
                            layer_2_size=32, 
                            layer_3_size=8, 
                            dropout=0.2,
                            convolution_kernel_size=3)

In [ ]:
# create optimizer with given learning rate 
opt = Adam()

# compile model
architecture.model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
# Train model on dataset
architecture.model.fit_generator(generator=data.generator_train,
                    validation_data=data.generator_valid,
                    use_multiprocessing=True,
                    workers=8)

## train features model

In [84]:
data = Data(sequence_length = 1, 
            return_CNN_features = True, 
            pretrained_model_name=pretrained_model_name,
            pooling = pooling)

2019-01-14 16:41:52,117 [MainThread  ] [INFO ]  Computing pretrained model features for video 1/25 using pretrained model: vgg16, pooling: max
2019-01-14 16:44:06,970 [MainThread  ] [INFO ]  Computing pretrained model features for video 2/25 using pretrained model: vgg16, pooling: max
2019-01-14 16:45:53,712 [MainThread  ] [INFO ]  Computing pretrained model features for video 3/25 using pretrained model: vgg16, pooling: max
2019-01-14 16:48:09,095 [MainThread  ] [INFO ]  Computing pretrained model features for video 4/25 using pretrained model: vgg16, pooling: max
2019-01-14 16:50:24,553 [MainThread  ] [INFO ]  Computing pretrained model features for video 5/25 using pretrained model: vgg16, pooling: max
2019-01-14 16:51:52,066 [MainThread  ] [INFO ]  Computing pretrained model features for video 6/25 using pretrained model: vgg16, pooling: max
2019-01-14 16:52:13,442 [MainThread  ] [INFO ]  Computing pretrained model features for video 7/25 using pretrained model: vgg16, pooling: max

(60597, 512) (60597, 8) 60597


In [86]:
architecture = Architecture(architecture="image_MLP_frozen", 
                            sequence_length=1,
                            num_classes = data.num_classes, 
                            frame_size = data.frame_size, 
                            pretrained_model_name='vgg16', 
                            pooling='max',
                            layer_1_size=64, 
                            layer_2_size=32, 
                            layer_3_size=8, 
                            dropout=0.2)

In [87]:
# train model
fit_history = train(architecture.model, data, path_model = pwd+'models/', learning_rate = 0.001, epochs = 30)

Train on 60597 samples, validate on 6416 samples
Epoch 1/30
60597/60597 [==============================] - 13s 211us/step - loss: 0.2564 - acc: 0.9110 - val_loss: 0.2061 - val_acc: 0.9262

Epoch 00001: val_acc improved from -inf to 0.92624, saving model to /mnt/seals/models/model.h5
Epoch 2/30
60597/60597 [==============================] - 12s 205us/step - loss: 0.1719 - acc: 0.9335 - val_loss: 0.2001 - val_acc: 0.9342

Epoch 00002: val_acc improved from 0.92624 to 0.93417, saving model to /mnt/seals/models/model.h5
Epoch 3/30
60597/60597 [==============================] - 13s 212us/step - loss: 0.1516 - acc: 0.9408 - val_loss: 0.1723 - val_acc: 0.9426

Epoch 00003: val_acc improved from 0.93417 to 0.94260, saving model to /mnt/seals/models/model.h5
Epoch 4/30
60597/60597 [==============================] - 12s 203us/step - loss: 0.1416 - acc: 0.9448 - val_loss: 0.1944 - val_acc: 0.9323

Epoch 00004: val_acc did not improve from 0.94260
Epoch 5/30
60597/60597 [==========================

KeyboardInterrupt: 

In [88]:
architecture.model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              (None, 64)                32832     
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 32)                2080      
_________________________________________________________________
dropout_2 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_4 (Dense)              (None, 8)                 264       
_________________________________________________________________
dropout_3 (Dropout)          (None, 8)                 0         
_________________________________________________________________
dense_5 (Dense)              (None, 8)                 72        
Total para

# Build cache

In [93]:
pretrained_model_names = ['inception_resnet_v2',
 'inception_v3',
 'mobilenetv2_1.00_224',
 'resnet50',
 'vgg16',
 'xception']

In [94]:
if __name__ == "__main__":
    
    logger.info("Building resized and feature cache")
    # build feature cache in advance by running python3 data.py
    for pretrained_model_name in pretrained_model_names:
        for pooling in poolings:
            logger.info("Building resized and feature cache: {}, {}".format(pretrained_model_name, pooling))
            data = Data(sequence_length=1, 
                        return_CNN_features=True,
                        pretrained_model_name = pretrained_model_name,
                        pooling=pooling)
            

    # build h5 cache
    logger.info("Building h5 cache")
    for sequence_length in [2,3,5,10,20]:
        for pretrained_model_name in pretrained_model_names:
            logger.info("Building h5 cache: {}, {}".format(sequence_length, pretrained_model_name))
            data = Data(sequence_length=sequence_length, 
                        return_CNN_features=False, 
                        pretrained_model_name = pretrained_model_name, 
                        return_generator = True,
                        verbose=True, batch_size=32)

2019-01-15 15:36:15,521 [MainThread  ] [INFO ]  Building resized and feature cache
2019-01-15 15:36:15,522 [MainThread  ] [INFO ]  Building resized and feature cache: inception_resnet_v2, max
2019-01-15 15:36:16,484 [MainThread  ] [INFO ]  Features already cached: /mnt/seals/cache/features/inception_resnet_v2/max/
2019-01-15 15:36:16,486 [MainThread  ] [INFO ]  Loading features data into memory [may take a few minutes]
2019-01-15 15:36:16,488 [MainThread  ] [INFO ]  Loading features data into memory: 1/25
2019-01-15 15:36:16,701 [MainThread  ] [INFO ]  Loading features data into memory: 2/25
2019-01-15 15:36:16,741 [MainThread  ] [INFO ]  Loading features data into memory: 3/25
2019-01-15 15:36:16,794 [MainThread  ] [INFO ]  Loading features data into memory: 4/25
2019-01-15 15:36:16,882 [MainThread  ] [INFO ]  Loading features data into memory: 5/25
2019-01-15 15:36:16,939 [MainThread  ] [INFO ]  Loading features data into memory: 6/25
2019-01-15 15:36:16,955 [MainThread  ] [INFO ]  L

(60597, 1536) (60597, 8) 60597


2019-01-15 15:36:18,797 [MainThread  ] [INFO ]  Building resized and feature cache: inception_resnet_v2, avg
2019-01-15 15:36:19,744 [MainThread  ] [INFO ]  Features already cached: /mnt/seals/cache/features/inception_resnet_v2/avg/
2019-01-15 15:36:19,745 [MainThread  ] [INFO ]  Loading features data into memory [may take a few minutes]
2019-01-15 15:36:19,748 [MainThread  ] [INFO ]  Loading features data into memory: 1/25
2019-01-15 15:36:19,804 [MainThread  ] [INFO ]  Loading features data into memory: 2/25
2019-01-15 15:36:19,829 [MainThread  ] [INFO ]  Loading features data into memory: 3/25
2019-01-15 15:36:19,917 [MainThread  ] [INFO ]  Loading features data into memory: 4/25
2019-01-15 15:36:20,005 [MainThread  ] [INFO ]  Loading features data into memory: 5/25
2019-01-15 15:36:20,060 [MainThread  ] [INFO ]  Loading features data into memory: 6/25
2019-01-15 15:36:20,074 [MainThread  ] [INFO ]  Loading features data into memory: 7/25
2019-01-15 15:36:20,154 [MainThread  ] [INFO

(60597, 1536) (60597, 8) 60597


2019-01-15 15:36:21,880 [MainThread  ] [INFO ]  Building resized and feature cache: inception_v3, max
2019-01-15 15:36:22,834 [MainThread  ] [INFO ]  Features already cached: /mnt/seals/cache/features/inception_v3/max/
2019-01-15 15:36:22,835 [MainThread  ] [INFO ]  Loading features data into memory [may take a few minutes]
2019-01-15 15:36:22,837 [MainThread  ] [INFO ]  Loading features data into memory: 1/25
2019-01-15 15:36:22,899 [MainThread  ] [INFO ]  Loading features data into memory: 2/25
2019-01-15 15:36:22,972 [MainThread  ] [INFO ]  Loading features data into memory: 3/25
2019-01-15 15:36:23,089 [MainThread  ] [INFO ]  Loading features data into memory: 4/25
2019-01-15 15:36:23,206 [MainThread  ] [INFO ]  Loading features data into memory: 5/25
2019-01-15 15:36:23,281 [MainThread  ] [INFO ]  Loading features data into memory: 6/25
2019-01-15 15:36:23,299 [MainThread  ] [INFO ]  Loading features data into memory: 7/25
2019-01-15 15:36:23,404 [MainThread  ] [INFO ]  Loading fe

...

Removing partially created sequences cache file: /mnt/seals/cache/sequences/resnet50/h5_meta.json


2019-01-15 16:09:26,310 [MainThread  ] [INFO ]  Computing frame sequence h5 files: /mnt/seals/cache/sequences/resnet50/ [may take a few minutes]
2019-01-15 16:09:26,312 [MainThread  ] [INFO ]  Computing frame sequence h5 files: 1/25 [precompute]
2019-01-15 16:09:29,536 [MainThread  ] [INFO ]  Computing frame sequence h5 files: 2/25 [precompute]
2019-01-15 16:09:32,181 [MainThread  ] [INFO ]  Computing frame sequence h5 files: 3/25 [precompute]
2019-01-15 16:09:35,574 [MainThread  ] [INFO ]  Computing frame sequence h5 files: 4/25 [precompute]
2019-01-15 16:09:38,950 [MainThread  ] [INFO ]  Computing frame sequence h5 files: 5/25 [precompute]
2019-01-15 16:09:41,179 [MainThread  ] [INFO ]  Computing frame sequence h5 files: 6/25 [precompute]
2019-01-15 16:09:41,741 [MainThread  ] [INFO ]  Computing frame sequence h5 files: 7/25 [precompute]
2019-01-15 16:09:44,712 [MainThread  ] [INFO ]  Computing frame sequence h5 files: 8/25 [precompute]
2019-01-15 16:09:48,116 [MainThread  ] [INFO ] 

Removing partially created sequences cache file: /mnt/seals/cache/sequences/xception/h5_meta.json


2019-01-15 16:16:44,358 [MainThread  ] [INFO ]  Computing frame sequence h5 files: 2/25 [precompute]
2019-01-15 16:16:46,834 [MainThread  ] [INFO ]  Computing frame sequence h5 files: 3/25 [precompute]
2019-01-15 16:16:49,777 [MainThread  ] [INFO ]  Computing frame sequence h5 files: 4/25 [precompute]
2019-01-15 16:16:52,688 [MainThread  ] [INFO ]  Computing frame sequence h5 files: 5/25 [precompute]
2019-01-15 16:16:54,740 [MainThread  ] [INFO ]  Computing frame sequence h5 files: 6/25 [precompute]
2019-01-15 16:16:55,364 [MainThread  ] [INFO ]  Computing frame sequence h5 files: 7/25 [precompute]
2019-01-15 16:16:57,886 [MainThread  ] [INFO ]  Computing frame sequence h5 files: 8/25 [precompute]
2019-01-15 16:17:00,839 [MainThread  ] [INFO ]  Computing frame sequence h5 files: 9/25 [precompute]
2019-01-15 16:17:03,743 [MainThread  ] [INFO ]  Computing frame sequence h5 files: 10/25 [precompute]
2019-01-15 16:17:04,442 [MainThread  ] [INFO ]  Computing frame sequence h5 files: 11/25 [

OSError: Can't prepare for writing data (file write failed: time = Tue Jan 15 16:24:42 2019
, filename = '/mnt/seals/cache/sequences/xception//h5_3train.h5', file descriptor = 114, errno = 28, error message = 'No space left on device', buf = 0x556afaacd79e, total write size = 77802, bytes this sub-write = 77802, bytes actually written = 18446744073709551615, offset = 48608813056)

# Train some initial models

In [116]:
pretrained_model_names

['inception_resnet_v2',
 'inception_v3',
 'mobilenetv2_1.00_224',
 'resnet50',
 'vgg16',
 'xception']

In [123]:
pretrained_model_names = ['vgg16']

In [124]:
layer_1_sizes = [512,256,128,64,32,16]
layer_2_sizes = [512,256,128,64,32,16]
layer_3_sizes = [512,256,128,64,32,16]
dropouts = [0.2]

# IMAGE_MLP_FROZEN
model_id = 0
for pretrained_model_name in pretrained_model_names:
    for layer_1_size in layer_1_sizes:
        for layer_2_size in layer_2_sizes:
            for layer_3_size in layer_3_sizes:
                for dropout in dropouts:
                    
                    data = Data(sequence_length = 1, 
                                return_CNN_features = True, 
                                pretrained_model_name=pretrained_model_name,
                                pooling = pooling)

                    architecture = Architecture(architecture="image_MLP_frozen", 
                                                sequence_length = 1,
                                                num_classes = data.num_classes, 
                                                frame_size = data.frame_size, 
                                                pretrained_model_name=pretrained_model_name, 
                                                pooling='max',
                                                layer_1_size=layer_1_size, 
                                                layer_2_size=layer_2_size, 
                                                layer_3_size=layer_3_size, 
                                                dropout=dropout)

                    path_model = pwd+'models/' + str(model_id) + '_'

                    # train model
                    fit_history = train(architecture.model, data, path_model = path_model, learning_rate = 0.001, epochs = 40)

                    results = {}
                    results['fit_val_acc'] = fit_history.history['val_acc'][-1]
                    results['fit_train_acc'] = fit_history.history['acc'][-1]
                    results['fit_val_loss'] = fit_history.history['val_loss'][-1]
                    results['fit_train_loss'] = fit_history.history['loss'][-1]
                    results['model_id'] = str(model_id)

                    results['pretrained_model_name'] = pretrained_model_name
                    results['layer_1_size'] = layer_1_size
                    results['layer_2_size'] = layer_2_size
                    results['layer_3_size'] = layer_3_size
                    results['dropout'] = dropout

                    with open(path_model + 'results.json', 'w') as fp:
                        json.dump(results, fp)
                        
                    model_id+=1

2019-01-15 22:39:46,940 [MainThread  ] [INFO ]  Features already cached: /mnt/seals/cache/features/vgg16/avg/
2019-01-15 22:39:46,942 [MainThread  ] [INFO ]  Loading features data into memory [may take a few minutes]
2019-01-15 22:39:46,944 [MainThread  ] [INFO ]  Loading features data into memory: 1/25
2019-01-15 22:39:46,981 [MainThread  ] [INFO ]  Loading features data into memory: 2/25
2019-01-15 22:39:46,997 [MainThread  ] [INFO ]  Loading features data into memory: 3/25
2019-01-15 22:39:47,017 [MainThread  ] [INFO ]  Loading features data into memory: 4/25
2019-01-15 22:39:47,044 [MainThread  ] [INFO ]  Loading features data into memory: 5/25
2019-01-15 22:39:47,056 [MainThread  ] [INFO ]  Loading features data into memory: 6/25
2019-01-15 22:39:47,075 [MainThread  ] [INFO ]  Loading features data into memory: 7/25
2019-01-15 22:39:47,092 [MainThread  ] [INFO ]  Loading features data into memory: 8/25
2019-01-15 22:39:47,110 [MainThread  ] [INFO ]  Loading features data into memo

(60597, 512) (60597, 8) 60597
Train on 60597 samples, validate on 6416 samples
Epoch 1/40
60597/60597 [==============================] - 28s 455us/step - loss: 0.1251 - acc: 0.9501 - val_loss: 0.1367 - val_acc: 0.9478

Epoch 00001: val_acc improved from -inf to 0.94777, saving model to /mnt/seals/models/model.h5
Epoch 2/40
60597/60597 [==============================] - 24s 402us/step - loss: 0.0995 - acc: 0.9597 - val_loss: 0.1418 - val_acc: 0.9472

Epoch 00002: val_acc did not improve from 0.94777
Epoch 3/40
60597/60597 [==============================] - 24s 398us/step - loss: 0.0911 - acc: 0.9630 - val_loss: 0.1588 - val_acc: 0.9357

Epoch 00003: val_acc did not improve from 0.94777
Epoch 4/40
60597/60597 [==============================] - 24s 401us/step - loss: 0.0853 - acc: 0.9654 - val_loss: 0.1501 - val_acc: 0.9418

Epoch 00004: val_acc did not improve from 0.94777
Epoch 5/40
60597/60597 [==============================] - 24s 399us/step - loss: 0.0811 - acc: 0.9670 - val_loss: 0.

2019-01-15 22:47:34,723 [MainThread  ] [INFO ]  Features already cached: /mnt/seals/cache/features/vgg16/avg/
2019-01-15 22:47:34,724 [MainThread  ] [INFO ]  Loading features data into memory [may take a few minutes]
2019-01-15 22:47:34,727 [MainThread  ] [INFO ]  Loading features data into memory: 1/25
2019-01-15 22:47:34,731 [MainThread  ] [INFO ]  Loading features data into memory: 2/25
2019-01-15 22:47:34,735 [MainThread  ] [INFO ]  Loading features data into memory: 3/25
2019-01-15 22:47:34,740 [MainThread  ] [INFO ]  Loading features data into memory: 4/25
2019-01-15 22:47:34,745 [MainThread  ] [INFO ]  Loading features data into memory: 5/25
2019-01-15 22:47:34,748 [MainThread  ] [INFO ]  Loading features data into memory: 6/25
2019-01-15 22:47:34,751 [MainThread  ] [INFO ]  Loading features data into memory: 7/25
2019-01-15 22:47:34,756 [MainThread  ] [INFO ]  Loading features data into memory: 8/25
2019-01-15 22:47:34,760 [MainThread  ] [INFO ]  Loading features data into memo

(60597, 512) (60597, 8) 60597
Train on 60597 samples, validate on 6416 samples
Epoch 1/40
60597/60597 [==============================] - 28s 456us/step - loss: 0.1255 - acc: 0.9498 - val_loss: 0.1419 - val_acc: 0.9502

Epoch 00001: val_acc improved from -inf to 0.95018, saving model to /mnt/seals/models/model.h5
Epoch 2/40
60597/60597 [==============================] - 24s 396us/step - loss: 0.0988 - acc: 0.9601 - val_loss: 0.1335 - val_acc: 0.9482

Epoch 00002: val_acc did not improve from 0.95018
Epoch 3/40
60597/60597 [==============================] - 24s 398us/step - loss: 0.0898 - acc: 0.9635 - val_loss: 0.1527 - val_acc: 0.9353

Epoch 00003: val_acc did not improve from 0.95018
Epoch 4/40
60597/60597 [==============================] - 24s 395us/step - loss: 0.0853 - acc: 0.9653 - val_loss: 0.1552 - val_acc: 0.9381

Epoch 00004: val_acc did not improve from 0.95018
Epoch 5/40
60597/60597 [==============================] - 24s 395us/step - loss: 0.0798 - acc: 0.9675 - val_loss: 0.

2019-01-15 22:54:32,036 [MainThread  ] [INFO ]  Features already cached: /mnt/seals/cache/features/vgg16/avg/
2019-01-15 22:54:32,038 [MainThread  ] [INFO ]  Loading features data into memory [may take a few minutes]
2019-01-15 22:54:32,040 [MainThread  ] [INFO ]  Loading features data into memory: 1/25
2019-01-15 22:54:32,044 [MainThread  ] [INFO ]  Loading features data into memory: 2/25
2019-01-15 22:54:32,048 [MainThread  ] [INFO ]  Loading features data into memory: 3/25
2019-01-15 22:54:32,052 [MainThread  ] [INFO ]  Loading features data into memory: 4/25
2019-01-15 22:54:32,057 [MainThread  ] [INFO ]  Loading features data into memory: 5/25
2019-01-15 22:54:32,060 [MainThread  ] [INFO ]  Loading features data into memory: 6/25
2019-01-15 22:54:32,063 [MainThread  ] [INFO ]  Loading features data into memory: 7/25
2019-01-15 22:54:32,067 [MainThread  ] [INFO ]  Loading features data into memory: 8/25
2019-01-15 22:54:32,072 [MainThread  ] [INFO ]  Loading features data into memo

...

2019-01-16 08:34:24,253 [MainThread  ] [INFO ]  Features already cached: /mnt/seals/cache/features/vgg16/avg/
2019-01-16 08:34:24,254 [MainThread  ] [INFO ]  Loading features data into memory [may take a few minutes]
2019-01-16 08:34:24,257 [MainThread  ] [INFO ]  Loading features data into memory: 1/25
2019-01-16 08:34:24,261 [MainThread  ] [INFO ]  Loading features data into memory: 2/25
2019-01-16 08:34:24,266 [MainThread  ] [INFO ]  Loading features data into memory: 3/25
2019-01-16 08:34:24,270 [MainThread  ] [INFO ]  Loading features data into memory: 4/25
2019-01-16 08:34:24,275 [MainThread  ] [INFO ]  Loading features data into memory: 5/25
2019-01-16 08:34:24,278 [MainThread  ] [INFO ]  Loading features data into memory: 6/25
2019-01-16 08:34:24,280 [MainThread  ] [INFO ]  Loading features data into memory: 7/25
2019-01-16 08:34:24,285 [MainThread  ] [INFO ]  Loading features data into memory: 8/25
2019-01-16 08:34:24,289 [MainThread  ] [INFO ]  Loading features data into memo

(60597, 512) (60597, 8) 60597
Train on 60597 samples, validate on 6416 samples
Epoch 1/40
60597/60597 [==============================] - 33s 538us/step - loss: 0.1324 - acc: 0.9475 - val_loss: 0.1423 - val_acc: 0.9499

Epoch 00001: val_acc improved from -inf to 0.94989, saving model to /mnt/seals/models/model.h5
Epoch 2/40
47520/60597 [======================>.......] - ETA: 5s - loss: 0.1060 - acc: 0.9573

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



60597/60597 [==============================] - 28s 457us/step - loss: 0.0773 - acc: 0.9684 - val_loss: 0.1417 - val_acc: 0.9505

Epoch 00008: val_acc did not improve from 0.95537
Epoch 9/40
60597/60597 [==============================] - 28s 459us/step - loss: 0.0757 - acc: 0.9690 - val_loss: 0.1445 - val_acc: 0.9498

Epoch 00009: val_acc did not improve from 0.95537
Epoch 10/40
59520/60597 [============================>.] - ETA: 0s - loss: 0.0737 - acc: 0.9698

KeyboardInterrupt: 

In [ ]:
# VIDEO_LRCNN_FROZEN
# IMAGE_MLP_FROZEN



layer_1_sizes = [512,256,128,64,32,16]
layer_2_sizes = [512,256,128,64,32,16]
layer_3_sizes = [512,256,128,64,32,16]
dropouts = [0.2]
sequence_model_layers = [1,2,3]
sequence_lengths = [3,5,10,20]
sequence_models = ['LSTM', 'SimpleRNN', 'GRU', 'Convolution1D']

# for pretrained_model_name in pretrained_model_names:
pretrained_model_name = 'vgg16'
for layer_1_size in layer_1_sizes:
    for layer_2_size in layer_2_sizes:
        for layer_3_size in layer_3_sizes:
            for dropout in dropouts:
                for sequence_model in sequence_models:
                    for sequence_model_layer in sequence_model_layers:
                        for sequence_length in sequence_lengths:

                            data = Data(sequence_length = sequence_length, 
                                        return_CNN_features = True, 
                                        pretrained_model_name=pretrained_model_name,
                                        pooling = pooling)

                            architecture = Architecture(architecture="video_LRCNN_frozen", 
                                                        sequence_model = 'LSTM',
                                                        sequence_model_layers = 1,
                                                        sequence_length = sequence_length,
                                                        num_classes = data.num_classes, 
                                                        frame_size = data.frame_size, 
                                                        pretrained_model_name='vgg16', 
                                                        pooling='max',
                                                        layer_1_size=64, 
                                                        layer_2_size=32, 
                                                        layer_3_size=8, 
                                                        dropout=0.2,
                                                        convolution_kernel_size=3)

                            path_model = pwd+'models/' + str(model_id) + '_'
                            
                            # train model
                            fit_history = train(architecture.model, data, path_model = path_model, learning_rate = 0.001, epochs = 40)
                            
                            results = {}
                            results['fit_val_acc'] = fit_history.history['val_acc'][-1]
                            results['fit_train_acc'] = fit_history.history['acc'][-1]
                            results['fit_val_loss'] = fit_history.history['val_loss'][-1]
                            results['fit_train_loss'] = fit_history.history['loss'][-1]
                            results['model_id'] = str(model_id)
                            
                            results['pretrained_model_name'] = pretrained_model_name
                            results['layer_1_size'] = layer_1_size
                            results['layer_2_size'] = layer_2_size
                            results['layer_3_size'] = layer_3_size
                            results['dropout'] = dropout
                            results['sequence_model'] = sequence_model
                            results['sequence_length'] = sequence_length
                            results['sequence_model_layer'] = sequence_model_layer

                            with open(path_model + 'results.json', 'w') as fp:
                                json.dump(results, fp)
                                
                            model_id+=1

In [144]:
data = Data(sequence_length = 1, 
            return_CNN_features = True, 
            pretrained_model_name=pretrained_model_name,
            pooling = pooling)

architecture = Architecture(architecture="image_MLP_frozen", 
                            sequence_length = 1,
                            num_classes = data.num_classes, 
                            frame_size = data.frame_size, 
                            pretrained_model_name=pretrained_model_name, 
                            pooling='max',
                            layer_1_size=64, 
                            layer_2_size=32, 
                            layer_3_size=16, 
                            dropout=dropout)

fit_history = train(architecture.model, data, path_model = path_model, learning_rate = 0.001, epochs = 40)

2019-01-16 08:46:32,464 [MainThread  ] [INFO ]  Features already cached: /mnt/seals/cache/features/vgg16/avg/
2019-01-16 08:46:32,465 [MainThread  ] [INFO ]  Loading features data into memory [may take a few minutes]
2019-01-16 08:46:32,468 [MainThread  ] [INFO ]  Loading features data into memory: 1/25
2019-01-16 08:46:32,473 [MainThread  ] [INFO ]  Loading features data into memory: 2/25
2019-01-16 08:46:32,477 [MainThread  ] [INFO ]  Loading features data into memory: 3/25
2019-01-16 08:46:32,482 [MainThread  ] [INFO ]  Loading features data into memory: 4/25
2019-01-16 08:46:32,486 [MainThread  ] [INFO ]  Loading features data into memory: 5/25
2019-01-16 08:46:32,490 [MainThread  ] [INFO ]  Loading features data into memory: 6/25
2019-01-16 08:46:32,493 [MainThread  ] [INFO ]  Loading features data into memory: 7/25
2019-01-16 08:46:32,498 [MainThread  ] [INFO ]  Loading features data into memory: 8/25
2019-01-16 08:46:32,502 [MainThread  ] [INFO ]  Loading features data into memo

(60597, 512) (60597, 8) 60597
Train on 60597 samples, validate on 6416 samples
Epoch 1/40
60597/60597 [==============================] - 33s 541us/step - loss: 0.1694 - acc: 0.9350 - val_loss: 0.1695 - val_acc: 0.9343

Epoch 00001: val_acc improved from -inf to 0.93434, saving model to /mnt/seals/models/model.h5
Epoch 2/40
60597/60597 [==============================] - 28s 463us/step - loss: 0.1271 - acc: 0.9506 - val_loss: 0.1524 - val_acc: 0.9441

Epoch 00002: val_acc improved from 0.93434 to 0.94407, saving model to /mnt/seals/models/model.h5
Epoch 3/40
60597/60597 [==============================] - 28s 463us/step - loss: 0.1156 - acc: 0.9549 - val_loss: 0.1749 - val_acc: 0.9306

Epoch 00003: val_acc did not improve from 0.94407
Epoch 4/40
60597/60597 [==============================] - 28s 464us/step - loss: 0.1091 - acc: 0.9577 - val_loss: 0.1614 - val_acc: 0.9368

Epoch 00004: val_acc did not improve from 0.94407
Epoch 5/40
60597/60597 [==============================] - 29s 471us/

# analyze results

In [ ]:
path_models = pwd + 'models/'

results = []

for folder, subs, files in os.walk(path_models):        
    for filename in files:
        if 'results.json' in filename:
            with open(os.path.abspath(os.path.join(folder, filename))) as f:
                data = json.load(f)
            results.append(data)

results = pd.DataFrame(results)        
results.sort_values("fit_val_acc", inplace=True, ascending=False)

In [142]:
results.head(10)

,dropout,fit_train_acc,fit_train_loss,fit_val_acc,fit_val_loss,layer_1_size,layer_2_size,layer_3_size,model_id,pretrained_model_name
38,0.2,0.971226,0.069500,0.952638,0.134846,256,64,32,58,vgg16
53,0.2,0.974475,0.061214,0.951761,0.168835,512,256,512,6,vgg16
33,0.2,0.971946,0.068300,0.951488,0.157837,512,128,16,17,vgg16
12,0.2,0.975558,0.059033,0.951488,0.154643,256,64,128,56,vgg16
50,0.2,0.971962,0.068007,0.950261,0.159397,512,256,16,11,vgg16
66,0.2,0.972552,0.067255,0.949306,0.143501,256,16,512,66,vgg16
0,0.2,0.972012,0.067722,0.948878,0.156286,512,64,16,23,vgg16
9,0.2,0.971412,0.069901,0.948819,0.149474,256,16,32,70,vgg16
74,0.2,0.977249,0.058907,0.947826,0.175842,512,16,64,33,vgg16
32,0.2,0.972045,0.068718,0.947767,0.171876,256,32,16,65,vgg16


In [143]:
results.tail(10)

,dropout,fit_train_acc,fit_train_loss,fit_val_acc,fit_val_loss,layer_1_size,layer_2_size,layer_3_size,model_id,pretrained_model_name
62,0.2,0.975151,0.061158,0.936428,0.198180,512,16,32,34,vgg16
72,0.2,0.972026,0.067342,0.936370,0.164086,256,128,256,49,vgg16
47,0.2,0.975968,0.058809,0.936370,0.168331,256,16,64,69,vgg16
69,0.2,0.972651,0.066966,0.935084,0.182124,512,32,32,28,vgg16
41,0.2,0.970487,0.073108,0.934987,0.188097,256,16,16,71,vgg16
55,0.2,0.972385,0.066343,0.934285,0.170115,256,256,512,42,vgg16
7,0.2,0.975663,0.058246,0.931811,0.228911,256,64,512,54,vgg16
26,0.2,0.972837,0.065532,0.929668,0.175158,512,64,64,21,vgg16
43,0.2,0.977093,0.054135,0.928480,0.199443,512,128,32,16,vgg16
39,0.2,0.974634,0.060648,0.926395,0.222550,512,256,128,8,vgg16
